In [1]:
import urllib
import zipfile
import os.path

import pandas as pd
from lxml import etree
import csv
import numpy as np

In [2]:
county_index = {"APPLING":221410, "ATKINSON":222414, "BACON":222415, "BAKER":222416, 
                "BALDWIN":222417, "BANKS":221414, "BARROW":222418, "BARTOW":222049, 
                "Ben_Hill":222419, "BERRIEN":222420, "BIBB":222421, "BLECKLEY":222422, 
                "BRANTLEY":222423, "BROOKS":222424, "BRYAN":222425, "BULLOCH":222426, 
                "BURKE":222427, "BUTTS":222428, "CALHOUN":222283, "CAMDEN":222429,
                "CANDLER":222028, "CARROLL":221938, "CATOOSA":222430, "CHARLTON":222431, 
                "CHATHAM":222168, "CHATTAHOOCHEE":222432, "CHATTOOGA":222434, 
                "CHEROKEE":222125, "CLARKE":221654, "CLAY":222135, "CLAYTON":222230, 
                "CLINCH":222173, "COBB":222156, "COFFEE":222435, "COLQUITT":222436, 
                "COLUMBIA":222237, "COOK":222437, "COWETA":222438, "CRAWFORD":222439, 
                "CRISP":222440, "DADE":221514, "DAWSON":222441, "DECATUR":222072, 
                "DEKALB":222190, "DODGE":222443, "DOOLY":222444, "DOUGHERTY":222153,
                "DOUGLAS":221801, "EARLY":222446, "ECHOLS":221752, "EFFINGHAM":221488, 
                "ELBERT":222447, "EMANUEL":222448, "EVANS":222449, "FANNIN":222450, 
                "FAYETTE":222129, "FLOYD":222451, "FORSYTH":222452, "FRANKLIN":222453, 
                "FULTON":221530, "GILMER":221898, "GLASCOCK":222454, "GLYNN":222171, 
                "GORDON":222455, "GRADY":222456, "GREENE":222457, "GWINNETT":222100, 
                "HABERSHAM":222458, "HALL":222460, "HANCOCK":222114, "HARALSON":222461, 
                "HARRIS":222130, "HART":222462, "HEARD":222463, "HENRY":222004,
                "HOUSTON":222464, "IRWIN":222466, "JACKSON":222467, "JASPER":222468, 
                "Jeff_Davis":222469, "JEFFERSON":222189, "JENKINS":222565, "JOHNSON":222566, 
                "JONES":222567, "LAMAR":222568, "LANIER":222038, "LAURENS":222570, 
                "LEE":222571, "LIBERTY":222572, "LINCOLN":222573, "LONG":221428, 
                "LOWNDES":221286, "LUMPKIN":221425, "MACON":222574, "MADISON":222575, 
                "MARION":222037, "MCDUFFIE":222576, "MCINTOSH":222577, "MERIWETHER":222578, 
                "MILLER":222580, "MITCHELL":222581, "MONROE":222582, "MONTGOMERY":222051,
                "MORGAN":221427, "MURRAY":222583, "MUSCOGEE":221666, "NEWTON":222584, 
                "OCONEE":222586, "OGLETHORPE":222071, "PAULDING":221608, "PEACH":222587, 
                "PICKENS":221993, "PIERCE":222588, "PIKE":222589, "POLK":222596, 
                "PULASKI":222597, "PUTNAM":222245, "QUITMAN":222598, "RABUN":222600, 
                "RANDOLPH":222601, "RICHMOND":222201, "ROCKDALE":222602, "SCHLEY":221485, 
                "SCREVEN":222604, "SEMINOLE":221764, "SPALDING":222608, "STEPHENS":221567, 
                "STEWART":221491, "SUMTER":222187, "TALBOT":222613, "TALIAFERRO":222614,
                "TATTNALL":221440, "TAYLOR":222615, "TELFAIR":222030, "TERRELL":222018, 
                "THOMAS":222617, "TIFT":222059, "TOOMBS":221981, "TOWNS":222618, 
                "TREUTLEN":222619, "TROUP":222620, "TURNER":222621, "TWIGGS":222622, 
                "UNION":222623, "UPSON":222624, "WALKER":222625, "WALTON":221627, 
                "WARE":222626, "WARREN":222627, "WASHINGTON":222628, "WAYNE":222629,
                "WEBSTER":221851, "WHEELER":222630, "WHITE":222631, "WHITFIELD":222632, 
                "WILCOX":222634, "WILKES":221510, "WILKINSON":221429, "WORTH":222635}

In [3]:
def write_total_ballots(elec, county):
    """
    Add rows to `total_ballots_by_precinct.csv`
    """
    # open a file for writing
    total_ballots_by_precinct = open('../../Data/County_votes/total_ballots_by_precinct.csv', 'a+')
    totals = elec.xpath("VoterTurnout/Precincts/Precinct")
    total = 0

    # create the csv writer object
    csvwriter = csv.writer(total_ballots_by_precinct)
    if county=="APPLING":
        csvwriter.writerow(["County", "Precinct_ID", "Ballots cast"])

    for v in totals:
        csvwriter.writerow([county, v.attrib["name"].lower().lstrip("0"), v.attrib["ballotsCast"]])
        total += int(v.attrib["ballotsCast"])

    # close the file
    total_ballots_by_precinct.close()

    # assert that the total by county, summed, equals the reported total
    assert total == int(elec.xpath("VoterTurnout/@ballotsCast")[0])
    return None

In [4]:
def write_precinct_votes(e_root, county):
    """
    Add rows to `votes_by_candidate_precinct.csv`
    """
    # open a file for writing
    votes = open('../../Data/County_votes/votes_by_candidate_precinct.csv', 'a+')

    # create the csv writer object
    csvwriter = csv.writer(votes)
    if county == "APPLING":
        csvwriter.writerow(["Contest", "Candidate", "Vote type", "County", "Precinct_ID", "Ballots cast"])

    for contest in e_root.iter("Contest"):
        contest_name = contest.attrib["text"]
        for choice in contest.iter("Choice"):
            candidate = choice.attrib["text"]
            for votetype in choice.iter("VoteType"):
                val = votetype.attrib["name"]
                for v in votetype:
                    csvwriter.writerow([contest_name, candidate, val, county, v.attrib["name"].lower().lstrip("0"), v.attrib["votes"]])

    # close the file
    votes.close()
    return None

In [5]:
index = 91641
#index2 = 221410 - 1
for county in county_index.keys():
    saveto = "../../Data/County_votes/" + county + ".zip"
    if not os.path.isfile(saveto):
        index2 = county_index[county]
        print("trying ", county, index, index2)
        
        # Download data from state website
        if county == "Ben_Hill" or county == "Jeff_Davis":
            county_str = county
        elif county == "DEKALB":
            county_str = "DeKalb"
        elif county == "MCDUFFIE":
            county_str = "McDuffie"
        elif county == "MCINTOSH":
            county_str = "McIntosh"
        else:
            county_str = str.capitalize(county)
        url = "https://results.enr.clarityelections.com/GA/" + county_str + "/" + \
            str(index) + "/" + str(index2) + "/reports/detailxml.zip"
        urllib.request.urlretrieve(url, saveto)
        print(county, "data successfully downloaded")

    # Data comes zipped. Unzip to detail.xml
    with zipfile.ZipFile(saveto,"r") as zip_ref:
        zip_ref.extractall("../../Data/County_votes/")
    
    # store some info about the xml file
    elec_fn = "../../Data/County_votes/detail.xml"
    elec = etree.parse(elec_fn)
    e_root = elec.getroot()
    
    # Write data from file
    write_total_ballots(elec, county)
    write_precinct_votes(e_root, county)
    
    # Increase the index number
    index += 1
    

# Calculate undervotes

In [6]:
total_ballots_cast = pd.read_csv('../../Data/County_votes/total_ballots_by_precinct.csv')
reported_votes = pd.read_csv('../../Data/County_votes/votes_by_candidate_precinct.csv')

In [7]:
reported_votes.head()

,Contest,Candidate,Vote type,County,Precinct_ID,Ballots cast
0,Governor,BRIAN KEMP (REP),Election Day,APPLING,1b,294
1,Governor,BRIAN KEMP (REP),Election Day,APPLING,1c,231
2,Governor,BRIAN KEMP (REP),Election Day,APPLING,2,188
3,Governor,BRIAN KEMP (REP),Election Day,APPLING,3a1,281
4,Governor,BRIAN KEMP (REP),Election Day,APPLING,3c,233


In [8]:
statewide_contests = np.array(["Governor", "Lieutenant Governor", "Secretary Of State", \
                               "Attorney General", "Commissioner Of Agriculture", \
                               "Commissioner Of Insurance", "State School Superintendent", "Commissioner Of Labor", \
                               "Public Service Commission, District 3 - Metro-Atlanta", \
                               "Public Service Commission, District 5 - Western"])
reported_votes = reported_votes[reported_votes["Contest"].isin(statewide_contests)]

reported_votes_by_contest = reported_votes.groupby(["Contest", "County", "Precinct_ID", "Vote type"])["Ballots cast"].agg(np.sum)
reported_votes_by_contest = reported_votes_by_contest.reset_index()
reported_votes_by_contest.head()

,Contest,County,Precinct_ID,Vote type,Ballots cast
0,Attorney General,APPLING,1b,Absentee by Mail,69
1,Attorney General,APPLING,1b,Advance in Person,362
2,Attorney General,APPLING,1b,Election Day,323
3,Attorney General,APPLING,1b,Provisional,2
4,Attorney General,APPLING,1c,Absentee by Mail,35


In [9]:
max_votes_estimated = reported_votes_by_contest.groupby(["County", "Precinct_ID", "Vote type"]).agg(np.max)
max_votes_estimated = max_votes_estimated.drop(columns=["Contest"]).reset_index()
max_votes_estimated.rename(columns={'Ballots cast': 'Total ballots'}, inplace=True)
max_votes_estimated.head()

,County,Precinct_ID,Vote type,Total ballots
0,APPLING,1b,Absentee by Mail,70
1,APPLING,1b,Advance in Person,375
2,APPLING,1b,Election Day,335
3,APPLING,1b,Provisional,2
4,APPLING,1c,Absentee by Mail,37


In [10]:
merged_votes = pd.DataFrame()
for contest in statewide_contests:
    this_race = reported_votes_by_contest["Contest"]==contest
    merged_votes_contest = pd.merge(max_votes_estimated, reported_votes_by_contest[this_race])
    merged_votes_contest["Undervotes"] = merged_votes_contest["Total ballots"] - merged_votes_contest["Ballots cast"]
    merged_votes = pd.concat([merged_votes, merged_votes_contest])
    
merged_votes.head()

,County,Precinct_ID,Vote type,Total ballots,Contest,Ballots cast,Undervotes
0,APPLING,1b,Absentee by Mail,70,Governor,70,0
1,APPLING,1b,Advance in Person,375,Governor,375,0
2,APPLING,1b,Election Day,335,Governor,335,0
3,APPLING,1b,Provisional,2,Governor,2,0
4,APPLING,1c,Absentee by Mail,37,Governor,37,0


In [11]:
merged_votes.to_csv('../../Data/County_votes/undervotes_by_precinct.csv', index=False)

In [12]:
# version information
%load_ext version_information
%version_information numpy, csv, pandas, lxml, notebook

Software versions
Python 3.7.1 64bit [Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 7.2.0
OS Darwin 18.0.0 x86_64 i386 64bit
numpy 1.15.4
csv 1.0
pandas 0.23.4
lxml 4.2.5
notebook 5.7.2
Sat Feb 02 11:58:20 2019 PST